In [1]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

/tmp/ipykernel_161669/4093896791.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")


In [2]:
llm.invoke('hi')

'Hello! How can I assist you today? If you have any questions or need information, feel free to ask.'

In [3]:
llm.invoke('سلام. خوبی؟')

'سلام! بله، خیلی خوبم، صبر کنید چطوری می\u200cشود؟ شما چطور هستید؟'

In [4]:
from langchain.chains import LLMMathChain

llm_math = LLMMathChain.from_llm(llm, verbose=True)

In [5]:
llm_math.prompt.template

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'

In [8]:
# llm_math.run("what is 13 raised to the .3432 power?")

In [6]:
i = 0
example_query = "What is 551368 divided by 82"
llm_math.invoke(example_query)



> Entering new LLMMathChain chain...
What is 551368 divided by 82To solve the problem of dividing 551368 by 82 using Python's `numexpr` library, we first need to translate the mathematical expression into a format that can be executed by `numexpr.evaluate`. Here’s how you can do it:

```text
551368 / 82
```

Now, let's evaluate this expression using `numexpr`:

...numexpr.evaluate("551368 / 82")...

The output of running this code will give us the result of the division.

Answer: 6722.0

So, 551368 divided by 82 is 6722.0.
> Finished chain.


{'question': 'What is 551368 divided by 82',
 'answer': 'Answer:  6722.0\n\nSo, 551368 divided by 82 is 6722.0.'}

## understand which formula is needed

In [2]:
from langchain.chains import LLMChain
from langchain.chains.router import LLMRouterChain
from langchain.prompts import PromptTemplate

In [ ]:
import json
import re
from langchain.schema import BaseOutputParser


In [3]:
# Define prompt templates for each formula chain
house_price_prompt = PromptTemplate(
    input_variables=["question"],
    template="This question asks about a house price. Apply Formula1: {question}"
)
other_task_prompt = PromptTemplate(
    input_variables=["question"],
    template="This question seems to be about another task. Apply Formula2: {question}"
)

In [4]:
# Create LLM chains for each formula
house_price_chain = LLMChain(llm=llm, prompt=house_price_prompt)
other_task_chain = LLMChain(llm=llm, prompt=other_task_prompt)

/tmp/ipykernel_161669/207616203.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  house_price_chain = LLMChain(llm=llm, prompt=house_price_prompt)


In [ ]:

def clean_keys(d: dict) -> dict:
    new_d = {}
    for key, value in d.items():
        # Strip whitespace and then remove surrounding quotes if present
        new_key = key.strip()
        if new_key.startswith('"') and new_key.endswith('"'):
            new_key = new_key[1:-1]
        if isinstance(value, dict):
            new_d[new_key] = clean_keys(value)
        else:
            new_d[new_key] = value
    return new_d

class CustomRouterOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # Extract the first JSON object from the text using regex
        match = re.search(r"(\{.*\})", text, re.DOTALL)
        if not match:
            raise ValueError("No JSON object found in text.")
        json_str = match.group(1)
        try:
            result = json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON: {json_str}") from e
        # Clean the keys of the parsed JSON
        result = clean_keys(result)
        # Validate required keys
        if "destination" not in result:
            raise ValueError("Output JSON is missing the 'destination' key.")
        if "next_inputs" not in result:
            raise ValueError("Output JSON is missing the 'next_inputs' key.")
        if not isinstance(result["next_inputs"], dict):
            raise ValueError("The 'next_inputs' key must be a dictionary.")
        if "question" not in result["next_inputs"]:
            raise ValueError("The 'next_inputs' dictionary must include the 'question' key.")
        return result

    @property
    def _type(self) -> str:
        return "custom_router_output_parser"


In [19]:
from langchain.chains.router.llm_router import LLMRouterChain
from langchain.prompts import PromptTemplate

router_output_parser = CustomRouterOutputParser()

router_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a routing assistant. Given the following question, decide which destination chain to use and return a valid JSON object with exactly two keys:\n"
        "1. destination: either \"house_price\" (if the question is about house pricing) or \"other_task\" for all other cases.\n"
        "2. next_inputs: a dictionary that includes the key \"question\" mapping to the original question text.\n"
        "For example, if the question is about house pricing, return:\n"
        "{{\"destination\": \"house_price\", \"next_inputs\": {{\"question\": \"What is the price of the house?\"}}}}\n"
        "Now, answer for the question: {question}"
    ),
    output_parser=router_output_parser  # your custom parser
)

router_chain = LLMRouterChain.from_llm(
    llm=llm,  # your routing LLM instance
    prompt=router_prompt,
    destination_chains={
        "house_price": house_price_chain,
        "other_task": other_task_chain,
    }
)


In [20]:
input_text = "please tell me about the price of the your house?"
result = router_chain({"question": input_text})
print(result)


{'question': 'please tell me about the price of the your house?', 'destination': 'house_price', 'next_inputs': {'question': 'please tell me about the price of your house?'}}


## New solution

In [5]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

/tmp/ipykernel_301528/4093896791.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")


In [6]:
import json
import re
from langchain.schema import BaseOutputParser
from langchain.chains.router.llm_router import LLMRouterChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate



In [ ]:
# Custom Output Parser Definition

def clean_keys(d: dict) -> dict:
    """Recursively clean dictionary keys by stripping whitespace and extra quotes."""
    new_d = {}
    for key, value in d.items():
        new_key = key.strip()
        if new_key.startswith('"') and new_key.endswith('"'):
            new_key = new_key[1:-1]
        if isinstance(value, dict):
            new_d[new_key] = clean_keys(value)
        else:
            new_d[new_key] = value
    return new_d

class CustomRouterOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # Extract the first JSON object from the text
        match = re.search(r"(\{.*\})", text, re.DOTALL)
        if not match:
            raise ValueError("No JSON object found in text.")
        json_str = match.group(1)
        try:
            result = json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON: {json_str}") from e
        # Clean the keys to remove any extra quotes
        result = clean_keys(result)
        # Validate required keys
        if "destination" not in result:
            raise ValueError("Output JSON is missing the 'destination' key.")
        if "next_inputs" not in result:
            raise ValueError("Output JSON is missing the 'next_inputs' key.")
        if not isinstance(result["next_inputs"], dict):
            raise ValueError("The 'next_inputs' key must be a dictionary.")
        if "question" not in result["next_inputs"]:
            raise ValueError("The 'next_inputs' dictionary must include the 'question' key.")
        return result

    @property
    def _type(self) -> str:
        return "custom_router_output_parser"

# Instantiate the custom output parser
router_output_parser = CustomRouterOutputParser()

In [ ]:

# Router Prompt: Including new formulas

router_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a routing assistant. Given the following question, decide which formula chain to use and return a valid JSON object with exactly two keys:\n"
        "1. destination: one of 'score_calculation', 'interest_calculation', or 'loan_calculation'.\n"
        "   - Use 'score_calculation' for questions about calculating customer score. \n"
        "   - Use 'interest_calculation' for questions about calculating deposit interest. \n"
        "   - Use 'loan_calculation' for questions about calculating remaining money for loans.\n"
        "2. next_inputs: a dictionary that includes the key 'question' mapping to the original question text.\n"
        "For example, if the question is 'How do I calculate the customer score?', return:\n"
        "{{\"destination\": \"score_calculation\", \"next_inputs\": {{\"question\": \"How do I calculate the customer score?\"}}}}\n"
        "Now, answer for the question: {question}"
    ),
    output_parser=router_output_parser
)


In [ ]:

# Create Formula Chains for Each Task/Formula


# Formula 1: Customer Score Calculation
score_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Customer Score Calculation:\n"
        "For the question: {question}, calculate the customer's score using the formula:\n"
        "    score = log((years_in_bank)^3) / age\n"
        "where 'years_in_bank' is the number of years the customer has had an account in our bank, and 'age' is the customer's age.\n"
        "Explain the calculation step by step."
    )
)
score_calculation_chain = LLMChain(llm=llm, prompt=score_calculation_prompt, verbose=True)

# Formula 2: Deposit Interest Calculation
interest_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Deposit Interest Calculation:\n"
        "For the question: {question}, calculate the deposit interest using the formula:\n"
        "    interest = ((deposit_amount * months) / 12) * 0.25\n"
        "where 'deposit_amount' is the amount of money deposited and 'months' is the number of months the money has been deposited.\n"
        "Explain the calculation step by step."
    )
)
interest_calculation_chain = LLMChain(llm=llm, prompt=interest_calculation_prompt, verbose=True)

# Formula 3: Loan Calculation Chain (unchanged)
loan_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Loan Calculation Expert:\n"
        "For the question: {question}, use the formula:\n"
        "    amount_to_pay = total_amount - (number_of_paid_loans * amount_per_loan)\n"
        "to calculate the amount of money required to pay all the loans.\n"
        "Explain the formula and provide an example calculation."
    )
)
loan_calculation_chain = LLMChain(llm=llm, prompt=loan_calculation_prompt, verbose=True)


/tmp/ipykernel_301528/3258733224.py:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  score_calculation_chain = LLMChain(llm=llm, prompt=score_calculation_prompt, verbose=True)


In [ ]:
# Create the Router Chain with the Three Destination Chains

destination_chains = {
    "score_calculation": score_calculation_chain,
    "interest_calculation": interest_calculation_chain,
    "loan_calculation": loan_calculation_chain,
}

router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt,
    destination_chains=destination_chains
)

In [ ]:
# Example Usage: Testing Different Questions


# 1. Customer Score Calculation Question (Formula 1)
input_text1 = "How do I calculate the customer score?"
router_result1 = router_chain({"question": input_text1})
print("Router output for customer score:", router_result1)
destination1 = router_result1["destination"]
next_inputs1 = router_result1["next_inputs"]
if destination1 in destination_chains:
    final_result1 = destination_chains[destination1](next_inputs1)
    print(f"Final result from '{destination1}':", final_result1)
else:
    print("Destination not found:", destination1)


/tmp/ipykernel_301528/522224383.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  router_result1 = router_chain({"question": input_text1})


Router output for customer score: {'question': 'How do I calculate the customer score?', 'destination': 'score_calculation', 'next_inputs': {'question': 'How do I calculate the customer score?'}}


> Entering new LLMChain chain...
Prompt after formatting:
Customer Score Calculation:
For the question: How do I calculate the customer score?, calculate the customer's score using the formula:
    score = log((years_in_bank)^3) / age
where 'years_in_bank' is the number of years the customer has had an account in our bank, and 'age' is the customer's age.
Explain the calculation step by step.

> Finished chain.
Final result from 'score_calculation': {'question': 'How do I calculate the customer score?', 'text': "To calculate a customer's score using the given formula:\n\n\\[ \\text{score} = \\frac{\\log((\\text{years\\_in\\_bank})^3)}{\\text{age}} \\]\n\nfollow these steps:\n\n### Step 1: Gather Input Data\n- **Years in Bank**: This is the number of years the customer has had an account with

In [13]:
# 2. Deposit Interest Calculation Question (Formula 2)
# input_text2 = "How do I calculate the deposit interest?"
input_text2 = "من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره"
router_result2 = router_chain({"question": input_text2})
print("Router output for deposit interest:", router_result2)
destination2 = router_result2["destination"]
print(f"destination2 {destination2}")
next_inputs2 = router_result2["next_inputs"]
print(f"next_input2 {next_inputs2}")
if destination2 in destination_chains:
    final_result2 = destination_chains[destination2](next_inputs2)
    print(f"destination_chain2 = {destination_chains[destination2]}")
    print(f"Final result from '{destination2}':", final_result2)
else:
    print("Destination not found:", destination2)

Router output for deposit interest: {'question': 'من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره', 'destination': 'interest_calculation', 'next_inputs': {'question': 'من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره'}}
destination2 interest_calculation
next_input2 {'question': 'من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره'}


> Entering new LLMChain chain...
Prompt after formatting:
Deposit Interest Calculation:
For the question: من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره, calculate the deposit interest using the formula:
    interest = ((deposit_amount * months) / 12) * 0.25
where 'deposit_amount' is the amount of money deposited and 'months' is the number of months the money has been deposited.
Explain the calculation step by step.

> Finished chain.
destination_chain2 = verbose=True prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="Deposit Interest Calculatio

In [ ]:
# 3. Loan Calculation Question (Loan Calculation)
input_text3 = "How much money should I pay to pay all my loans to the bank?"
router_result3 = router_chain({"question": input_text3})
print("Router output for loan calculation:", router_result3)
destination3 = router_result3["destination"]
next_inputs3 = router_result3["next_inputs"]
if destination3 in destination_chains:
    final_result3 = destination_chains[destination3](next_inputs3)
    print(f"Final result from '{destination3}':", final_result3)
else:
    print("Destination not found:", destination3)